In [37]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tqdm
import ray
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from scipy import sparse
import logging
from mol2vec.features import mol2alt_sentence
from gensim.models.word2vec import Word2Vec

In [18]:
ray.init()

2021-05-26 22:38:18,136	INFO services.py:1267 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '172.31.34.223',
 'raylet_ip_address': '172.31.34.223',
 'redis_address': '172.31.34.223:20179',
 'object_store_address': '/tmp/ray/session_2021-05-26_22-38-15_934524_25070/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-26_22-38-15_934524_25070/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2021-05-26_22-38-15_934524_25070',
 'metrics_export_port': 54370,
 'node_id': 'e0e172425193445f1a039e5786930123a8924d351c32ba927c5f8df1'}

In [4]:
RECEPTOR = "EnamineHTS"
INPUT_DATA_DIR = "/mnt/efs/enamine"
INPUT_DATA = f"{INPUT_DATA_DIR}/{RECEPTOR}_scores.csv"
# INPUT_SMILES = f"{INPUT_DATA_DIR}/{RECEPTOR}.smi"
# OUTPUT_EMBEDDINGS = f"{INPUT_DATA_DIR}/{RECEPTOR}_embeddings.csv"
OUTPUT_DATA_DIR = "../processed_data"

In [5]:
def get_data():
    ligands_df = pd.read_csv(INPUT_DATA)
    
    return ligands_df

In [19]:
word2vec_model = Word2Vec.load("/home/ubuntu/mol2vec/examples/models/model_300dim.pkl")
word2vec_model.wv.init_sims()

<ipython-input-19-27f0ea2b60c3>:2: DeprecationWarning: Call to deprecated `init_sims` (Use fill_norms() instead. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  word2vec_model.wv.init_sims()


In [23]:
UNCOMMON = "UNK"
# word2vec_model.wv[UNCOMMON]

# Embed molecules

In [25]:
# mol2vec featurize -i "/mnt/efs/enamine/EnamineHTS.smi" -o "/mnt/efs/enamine/EnamineHTS_embeddings.csv" -m examples/models/model_300dim.pkl -r 1 --uncommon UNK
# features.featurize(in_file=INPUT_SMILES, 
#                    out_file=OUTPUT_EMBEDDINGS, 
#                    model_path="/home/ubuntu/mol2vec/examples/models/model_300dim.pkl", 
#                    r=1, 
#                    uncommon="UNK")

In [41]:
def sentences2vec(sentences, model, unseen=None):
    """Generate vectors for each sentence (list) in a list of sentences. Vector is simply a
    sum of vectors for individual words.
    
    Parameters
    ----------
    sentences : list, array
        List with sentences
    model : word2vec.Word2Vec
        Gensim word2vec model
    unseen : None, str
        Keyword for unseen words. If None, those words are skipped.
        https://stats.stackexchange.com/questions/163005/how-to-set-the-dictionary-for-text-analysis-using-neural-networks/163032#163032

    Returns
    -------
    np.array
    """
    keys = set(model.wv.key_to_index)
    vec = []
    
    if unseen:
        unseen_vec = model.wv.get_vector(unseen)

    for sentence in sentences:
        if unseen:
            vec.append(sum([model.wv.get_vector(y) if y in set(sentence) & keys
                       else unseen_vec for y in sentence]))
        else:
            vec.append(sum([model.wv.get_vector(y) for y in sentence 
                            if y in set(sentence) & keys]))
    return np.array(vec, dtype=np.float32)

In [33]:
@ray.remote
def create_mol_sentence(smiles, score, r, i):
    if i % 10000 == 0:
        logging.basicConfig(level=logging.INFO)
        logging.info(i)
        
    mol = Chem.MolFromSmiles(smiles)
    # smiles = Chem.MolToSmiles(mol)
    
    if not mol:
        return
    
    alt_sentence = mol2alt_sentence(mol, radius=r)
    
    return alt_sentence, score

In [34]:
def get_embeddings(ligands_df, model, radius=1):
    future_values = [create_mol_sentence.remote(smiles=smiles, score=score, r=radius, i=i) for (i, (smiles, score)) in enumerate(zip(ligands_df["smiles"], ligands_df["score"]))]
    
    values = [v for v in ray.get(future_values) if v]
    mol_sentences, scores = zip(*values)

#     vectors = sentences2vec(sentences=mol_sentences, model=model, unseen=UNCOMMON)

    return mol_sentences, scores

In [35]:
ligands_df = get_data()

In [36]:
embeddings, scores = get_embeddings(ligands_df=ligands_df, model=word2vec_model, radius=1)

(pid=25786) INFO:root:0
(pid=25785) INFO:root:10000
(pid=25785) INFO:root:20000
(pid=25777) INFO:root:30000
(pid=25776) INFO:root:40000
(pid=25781) INFO:root:50000
(pid=25784) INFO:root:60000
(pid=25780) INFO:root:70000
(pid=25776) INFO:root:80000
(pid=25787) INFO:root:90000
(pid=25783) INFO:root:100000
(pid=25775) INFO:root:110000
(pid=25785) INFO:root:120000
(pid=25784) INFO:root:130000
(pid=25784) INFO:root:140000
(pid=25786) INFO:root:150000
(pid=25779) INFO:root:160000
(pid=25781) INFO:root:170000
(pid=25781) INFO:root:180000
(pid=25783) INFO:root:190000
(pid=25780) INFO:root:200000
(pid=25778) INFO:root:210000
(pid=25784) INFO:root:220000
(pid=25778) INFO:root:230000
(pid=25776) INFO:root:240000
(pid=25784) INFO:root:250000
(pid=25788) INFO:root:260000
(pid=25783) INFO:root:270000
(pid=25789) INFO:root:280000
(pid=25787) INFO:root:290000
(pid=25782) INFO:root:300000
(pid=25781) INFO:root:310000
(pid=25786) INFO:root:320000
(pid=25777) INFO:root:330000
(pid=25787) INFO:root:340000

(pid=25780) INFO:root:1940000
(pid=25775) INFO:root:1950000
(pid=25789) INFO:root:1960000
(pid=25780) INFO:root:1970000
(pid=25788) INFO:root:1980000
(pid=25785) INFO:root:1990000
(pid=25778) INFO:root:2000000
(pid=25782) INFO:root:2010000
(pid=25787) INFO:root:2020000
(pid=25781) INFO:root:2030000
(pid=25783) INFO:root:2040000
(pid=25787) INFO:root:2050000
(pid=25777) INFO:root:2060000
(pid=25784) INFO:root:2070000
(pid=25776) INFO:root:2080000
(pid=25789) INFO:root:2090000
(pid=25774) INFO:root:2100000


In [39]:
vectors = sentences2vec(sentences=embeddings, model=word2vec_model, unseen=UNCOMMON)

In [42]:
np.save(f"{INPUT_DATA_DIR}/{RECEPTOR}_embeddings.npy", vectors)
np.save(f"{INPUT_DATA_DIR}/{RECEPTOR}_embedding_scores.npy", np.array(scores))